In [1]:
import pandas as pd

## Register Status

In [2]:
def ini_register_status():
    register_status = {}

    # Registradores inteiros (x0..x31)
    for i in range(32):
        register_status[f"x{i}"] = {"x": i, "x_type": "int", "writer": None}

    # Registradores de ponto flutuante (f0..f31)
    for i in range(32):
        register_status[f"f{i}"] = {"f": i, "f_type": "float", "writer": None}
    return register_status


## Parser das instruções

In [3]:
OPCODES = {
    'fld': 0,
    'fsd': 1,
    'fadd': 2,
    'fsub': 3,
    'fmul': 4,
    'fdiv': 5
}

# Define register prefix constants
REG_PREFIXES = {
    'x': 'int',
    'f': 'float'
}

In [4]:
def init_instruction_status(program_name):
    instruction_status =[]
    with open(program_name, "r") as file:
        for i, line in enumerate(file):
            op = OPCODES[line.split()[0]]
            if (op == 0 or op==1):
                rd = line.split()[1].replace(",", "")
                rs1 = line.split()[2].split("(")[1].replace(")", "")
                rs2 = None
                imm = line.split()[2].split("(")[0]
                fu_type = 0
            else:
                if op == 2 or op == 3: fu_type = 2
                elif op == 4: fu_type = 1
                elif op == 5: fu_type = 3
                rd = line.split()[1].replace(",", "")
                rs1 = line.split()[2].replace(",", "")
                rs2 = line.split()[3].replace(",", "")
                imm = None          
            instruction_status.append({ 
                "id": i,
                "inst": line.replace("\n", ""),
                "opcode": op,
                "fu_name": None,
                "fu_type": fu_type,
                "rd": rd,
                "rs1": rs1,
                "rs2": rs2,
                "imm": imm,
                "issue": None,
                "read_operands": None,
                "execution_complete": None,
                "write_result": None

            })
    return  instruction_status

## Parser das functionals units

In [5]:
def parser_fus_configs(config_name):
    fus_configs = {}
    with open(config_name, 'r') as f:
        linhas = f.readlines()
        for linha in linhas:
            partes = linha.strip().split()
            fus_configs[partes[0]] = {'qtd': partes[1], 'cycles': partes[2]}
    return fus_configs

In [6]:
def init_fus_status(fus_configs):
    fus_configs = parser_fus_configs(config_name="tests/uf_config1.in")
    fus_status = {}
    for uf_type, config in fus_configs.items():
        if uf_type == 'int':
            op = 0
        elif uf_type == 'mult':
            op = 1
        elif uf_type == 'add':
            op = 2
        elif uf_type == 'div':
            op = 3
        for c in range(int(config['qtd'])):
            fus_status[f'{uf_type}{c+1}'] = {
                'Opcode': op, 
                'Busy': False, 
                'Op': None, 
                'Fi': None,
                'Fj': None, 
                'Fk': None, 
                'Qj': None, 
                'Qk': None,
                'Rj': False, 
                'Rk': False, 
                'Cycles_left': int(config['cycles']), 
                'Cycles': int(config['cycles'])
            }
    return fus_status

## ISSUE

Verificar se a unidade funcional requerida está livre -- sem hazards estruturais
    
    Functional unit status -- verificada aqui

Verificar se nenhuma outra instrução vai escrever no rd -- sem WAW hazards
    
    Register Status -- verificado aqui

In [7]:
def issue(instr, fus_status, register_status):
    opcode_i = instr['opcode']
    id_i = instr['id']
    rs1_i = instr['rs1']
    rs2_i = instr['rs2']
    rd_i = instr['rd']
    for fu_name, fu in fus_status.items():
        # verifica se a unidade funcional corresponde à instrução
        # se a unidade funcional não está vazia
        # se nenhuma outra FU n está escrevendo no rd
        if fu['Opcode'] != opcode_i or fu['Busy'] or register_status[rd_i]['writer'] is not None:
            continue
        fu['Busy'] = True
        fu['Op'] = id_i
        fu['Fi'] = rd_i
        fu['Fj'] = rs1_i
        fu['Fk'] = rs2_i
        fu['Qj'] = register_status[rs1_i]['writer'] if rs1_i is not None else None
        fu['Qk'] = register_status[rs2_i]['writer'] if rs2_i is not None else None
        fu['Rj'] = fu['Qj'] is None
        fu['Rk'] = fu['Qk'] is None
        register_status[rd_i]['writer'] = fu_name
        instr['fu_name'] = fu_name
        return True
    return False

In [54]:
def issue(instr, fus_status, register_status):
    opcode_i = instr['opcode']
    id_i = instr['id']
    rs1_i = instr['rs1']
    rs2_i = instr['rs2']
    rd_i = instr['rd']
    fu_type = instr['fu_type']

    print(f"\n[ISSUE] Tentando emitir instrução {id_i}: {instr['inst']}")

    for fu_name, fu in fus_status.items():
        print(f"  Checando FU {fu_name} (Opcode esperado={fu_type}, FU Opcode={fu['Opcode']}, Busy={fu['Busy']})")

        # 1. opcode não bate
        if fu['Opcode'] != fu_type:
            print(f"    -> Falha: FU {fu_name} não suporta opcode {fu_type}")
            continue

        # 2. FU já ocupada
        if fu['Busy']:
            print(f"    -> Falha: FU {fu_name} ocupada (Busy=True)")
            continue

        # 3. registrador destino já tem writer
        if register_status[rd_i]['writer'] is not None:
            print(f"    -> Falha: registrador {rd_i} já tem writer {register_status[rd_i]['writer']}")
            return False

        # Se passou em tudo, ocupa FU
        print(f"    -> Sucesso: Instr {id_i} emitida na FU {fu_name}")
        fu['Busy'] = True
        fu['Op'] = id_i
        fu['Fi'] = rd_i
        fu['Fj'] = rs1_i
        fu['Fk'] = rs2_i
        fu['Qj'] = register_status[rs1_i]['writer'] if rs1_i is not None else None
        fu['Qk'] = register_status[rs2_i]['writer'] if rs2_i is not None else None
        fu['Rj'] = fu['Qj'] is None
        fu['Rk'] = fu['Qk'] is None
        register_status[rd_i]['writer'] = fu_name
        instr['fu_name'] = fu_name
        return True

    print(f"  -> Nenhuma FU disponível para emitir instrução {id_i}")
    return False


## Read Operands

verifica no `register_status` se alguma unidade funcional vai escrever nos operandos de leitura `rs1` e `rs2`

In [ ]:
def read_operands(rs1_i, rs2_i, register_status):
    ready_rs1 = (rs1_i is None) or (register_status[rs1_i]['writer'] is None)
    ready_rs2 = (rs2_i is None) or (register_status[rs2_i]['writer'] is None)
    return ready_rs1 and ready_rs2

## Complete Execution

In [10]:

def execute(fu, cycle, inst):
    if fu['Cycles_left'] > 0:
        fu['Cycles_left'] -= 1
        print(f"INFO: Instrução '{inst['inst']}' está executando. Ciclos restantes: {fu['Cycles_left']}.")
    if fu['Cycles_left'] == 0:
        inst['execution_complete'] = cycle
        fu['Cycles_left'] = fu['Cycles']
        print(f"OK: Instrução '{inst['inst']}' completou a execução.")

## Write Results

In [78]:

def write_results(fus_status, inst, cycle, register_status):
    inst['write_result'] = cycle
    fu = fus_status[inst['fu_name']]
    print(f"OK: Instrução '{inst['inst']}' fez Write Back.")
    if fu['Fi'] and register_status[fu['Fi']]['writer'] == inst['fu_name']:
        register_status[fu['Fi']]['writer'] = None
        print(f"Liberando registrador de destino {fu['Fi']}")

    # Libera a Unidade Funcional, limpando seu estado
    fu.update({
        'Busy': False, 'Op': None, 'Fi': None, 'Fj': None, 'Fk': None,
        'Qj': None, 'Qk': None, 'Rj': False, 'Rk': False
    })


In [81]:
def write_results(fus_status, inst, cycle, register_status):
    """
    Realiza o Write Back, mas APENAS se não houver um hazard WAR.
    """
    fu = fus_status[inst['fu_name']]
    fi_register = fu['Fi'] # O registrador de destino desta instrução

    # --- VERIFICAÇÃO DE HAZARD WAR ---
    # Itera sobre todas as unidades funcionais para ver se alguma precisa ler nosso registrador de destino.
    for other_fu_name, other_fu in fus_status.items():
        # Ignora a verificação contra si mesma
        if other_fu_name == inst['fu_name']:
            continue

        # Condição de Hazard: Outra FU ainda não leu (Rj=False) e seu operando fonte (Fj)
        # é o nosso registrador de destino (Fi).
        if not other_fu['Rj'] and other_fu['Fj'] == fi_register:
            print(f"STALL (WAR): Write Back de '{inst['inst']}' pausado. FU '{other_fu_name}' ainda precisa ler {fi_register}.")
            return # Stall! Não faz nada neste ciclo.

        # Repete a verificação para o segundo operando (Fk)
        if not other_fu['Rk'] and other_fu['Fk'] == fi_register:
            print(f"STALL (WAR): Write Back de '{inst['inst']}' pausado. FU '{other_fu_name}' ainda precisa ler {fi_register}.")
            return # Stall!

    # --- Se passou na verificação (sem hazards), procede com o Write Back ---
    inst['write_result'] = cycle
    print(f"OK: Instrução '{inst['inst']}' fez Write Back.")

    # Libera o registrador de destino, se ele pertencer a esta FU
    if fi_register and register_status[fi_register]['writer'] == inst['fu_name']:
        register_status[fi_register]['writer'] = None
        print(f"Liberando registrador de destino {fi_register}")

    # Libera a Unidade Funcional, limpando seu estado
    fu.update({
        'Busy': False, 'Op': None, 'Fi': None, 'Fj': None, 'Fk': None,
        'Qj': None, 'Qk': None, 'Rj': False, 'Rk': False
    })

## Scoreboard

### Montando a instrução novamente

In [57]:
prog_file = "tests/ex3.s"
fus_file = "tests/uf_config3.in"

In [17]:
def has_pending_instructions(inst_status):
    return any(inst['write_result'] is None for inst in inst_status)


fld f1, 0(x1)
fld f5, 0(x1)

In [41]:
register_status = ini_register_status()
instruction_status = init_instruction_status(prog_file)
fus_status = init_fus_status(fus_file)

Debug

In [84]:
fus_status

{'int1': {'Opcode': 0,
  'Busy': False,
  'Op': None,
  'Fi': None,
  'Fj': None,
  'Fk': None,
  'Qj': None,
  'Qk': None,
  'Rj': False,
  'Rk': False,
  'Cycles_left': 1,
  'Cycles': 1},
 'int2': {'Opcode': 0,
  'Busy': False,
  'Op': None,
  'Fi': None,
  'Fj': None,
  'Fk': None,
  'Qj': None,
  'Qk': None,
  'Rj': False,
  'Rk': False,
  'Cycles_left': 1,
  'Cycles': 1},
 'mult1': {'Opcode': 1,
  'Busy': False,
  'Op': None,
  'Fi': None,
  'Fj': None,
  'Fk': None,
  'Qj': None,
  'Qk': None,
  'Rj': False,
  'Rk': False,
  'Cycles_left': 4,
  'Cycles': 4},
 'mult2': {'Opcode': 1,
  'Busy': False,
  'Op': None,
  'Fi': None,
  'Fj': None,
  'Fk': None,
  'Qj': None,
  'Qk': None,
  'Rj': False,
  'Rk': False,
  'Cycles_left': 4,
  'Cycles': 4},
 'add1': {'Opcode': 2,
  'Busy': True,
  'Op': 5,
  'Fi': 'f6',
  'Fj': 'f3',
  'Fk': 'f4',
  'Qj': 'div1',
  'Qk': None,
  'Rj': False,
  'Rk': True,
  'Cycles_left': 2,
  'Cycles': 2},
 'div1': {'Opcode': 3,
  'Busy': True,
  'Op': 4,
 

In [85]:
register_status = ini_register_status()
instruction_status = init_instruction_status(prog_file)
fus_status = init_fus_status(fus_file)
pc = 0
cycle = 1

while pc < len(instruction_status) or has_pending_instructions(instruction_status):
    print(f"\n--- Cycle {cycle} ---")
    print(f"PC = {pc}")
    liberar_write = False
    # WRITE RESULTS
    for inst in instruction_status:
        if inst['execution_complete'] is not None and inst['execution_complete'] < cycle and inst['write_result'] is None:
            write_results(fus_status, inst, cycle, register_status)
            liberar_write = True
    for inst in instruction_status:
        if inst['read_operands'] is not None and inst['read_operands'] < cycle and inst['execution_complete'] is None:
            fu = fus_status[inst['fu_name']]
            execute(fu, cycle, inst)

    # 1. Read operands
    for inst in instruction_status:
        if inst['issue'] is not None and inst['read_operands'] is None:
            fu = fus_status[inst['fu_name']]
            if not fu['Rj']:
                writer_j = register_status.get(fu['Fj'], {}).get('writer')
                if writer_j is None:
                    fu['Rj'] = True
                    fu['Qj'] = None
                else:
                    writer_inst = next((i for i in instruction_status if i['fu_name'] == writer_j), None)
                    if writer_inst and writer_inst['execution_complete'] is not None:
                        fu['Rj'] = True
                        fu['Qj'] = None
            if not fu['Rk']:
                if not fu['Fk']: # nao tem Fk
                    fu['Rk'] = True
                else:
                    writer_k = register_status.get(fu['Fk'], {}).get('writer')
                    if writer_k is None:
                        fu['Rk'] = True
                        fu['Qk'] = None
            if fu['Rj'] and fu['Rk']:
                inst['read_operands'] = cycle
                print(f"OK: Instrução '{inst['inst']}' leu os operandos.")
    

    # 2. Issue
    if pc < len(instruction_status):
        if not liberar_write:
            next_instruction = instruction_status[pc]
            if issue(next_instruction, fus_status, register_status):
                next_instruction['issue'] = cycle
                print(f"--> Instr {next_instruction['id']} emitida")
                pc += 1
            else:
                print(f"--> Instr {next_instruction['id']} NÃO pôde ser emitida")
                # print(issue(next_instruction, fus_status, register_status))

    cycle += 1
    
    if cycle > 33: # Cláusula de segurança
        print("\nSimulação parada após 30 ciclos para evitar loop infinito.")
        break



--- Cycle 1 ---
PC = 0

[ISSUE] Tentando emitir instrução 0: fld  f1, 100(x7)
  Checando FU int1 (Opcode esperado=0, FU Opcode=0, Busy=False)
    -> Sucesso: Instr 0 emitida na FU int1
--> Instr 0 emitida

--- Cycle 2 ---
PC = 1
OK: Instrução 'fld  f1, 100(x7)' leu os operandos.

[ISSUE] Tentando emitir instrução 1: fmul f2, f2, f4
  Checando FU int1 (Opcode esperado=1, FU Opcode=0, Busy=True)
    -> Falha: FU int1 não suporta opcode 1
  Checando FU int2 (Opcode esperado=1, FU Opcode=0, Busy=False)
    -> Falha: FU int2 não suporta opcode 1
  Checando FU mult1 (Opcode esperado=1, FU Opcode=1, Busy=False)
    -> Sucesso: Instr 1 emitida na FU mult1
--> Instr 1 emitida

--- Cycle 3 ---
PC = 2
INFO: Instrução 'fld  f1, 100(x7)' está executando. Ciclos restantes: 0.
OK: Instrução 'fld  f1, 100(x7)' completou a execução.
OK: Instrução 'fmul f2, f2, f4' leu os operandos.

[ISSUE] Tentando emitir instrução 2: fadd f2, f1, f3
  Checando FU int1 (Opcode esperado=2, FU Opcode=0, Busy=True)
    

## Print Resultado

In [61]:
import pandas as pd

def formatar_tabela_scoreboard(instruction_status):
    """
    Formata o resultado da simulação do scoreboard em uma tabela legível.

    Args:
        instruction_status (list): A lista final com o status de cada instrução.

    Returns:
        pandas.DataFrame: A tabela formatada com os resultados.
    """
    # 1. Cria o DataFrame a partir da sua lista de resultados
    df = pd.DataFrame(instruction_status)

    # 2. Seleciona apenas as colunas que importam para a tabela final
    tabela_final = df[['inst', 'issue', 'read_operands', 'execution_complete', 'write_result']]

    # 3. Renomeia as colunas para o formato desejado
    tabela_final = tabela_final.rename(columns={
        'inst': 'Instruction/Cicle',
        'issue': 'Issue',
        'read_operands': 'Read',
        'execution_complete': 'Execute', # Este é o ciclo em que a execução TERMINA
        'write_result': 'Write'
    })

    # 4. Substitui valores 'None' ou 'NaN' por um traço para ficar mais limpo
    tabela_final = tabela_final.fillna("-")

    return tabela_final

In [86]:
tabela_de_resultados = formatar_tabela_scoreboard(instruction_status)
print(tabela_de_resultados.to_string())

  Instruction/Cicle  Issue  Read  Execute  Write
0  fld  f1, 100(x7)      1     2        3      4
1   fmul f2, f2, f4      2     3        7      8
2   fadd f2, f1, f3      9    10       12     13
3    fld  f9, 0(x3)     10    11       12     13
4   fdiv f3, f1, f7     11    12       22     23
5   fsub f6, f3, f4     14    22       24     25
6   fmul f7, f1, f2     15    16       20     21
7   fadd f4, f5, f2     26    27       29     30
8  fsd  f1, 50(x11)     27    28       29     30
